In [2]:
import pandas as pd
import numpy as np
import dask #used here to scale computing capabilities of pandas through parallelism
import dask.dataframe as dd
import pickle #since openning the data was time consuming let's just save it in a pickle file

In [3]:
crsp_pickle=open ("crsp.pickle","rb")
ibes_pickle=open ("ibes.pickle","rb")
comp_pickle=open ("comp.pickle","rb")

In [4]:
crsp=pickle.load(crsp_pickle)
crsp.info

<bound method DataFrame.info of Dask DataFrame Structure:
               Unnamed: 0 PERMNO    date    SHRCD   EXCHCD  NCUSIP  TICKER   CUSIP   DIVAMT    FACPR   NWPERM    BIDLO    ASKHI      PRC      VOL      RET      BID      ASK   SHROUT   SPREAD     RETX
npartitions=12                                                                                                                                                                                         
                    int64  int64  object  float64  float64  object  object  object  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64
                      ...    ...     ...      ...      ...     ...     ...     ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...
...                   ...    ...     ...      ...      ...     ...     ...     ...      ...      ...      ...      ...      ...      ...      

In [5]:
ibes=pickle.load(ibes_pickle)
ibes.info

<bound method DataFrame.info of Dask DataFrame Structure:
               Unnamed: 0  TICKER   CUSIP ESTIMATOR ANALYS    FPI    VALUE FPEDATS REVDATS REVTIMS ANNDATS ANNTIMS   ACTUAL ANNDATS_ACT ANNTIMS_ACT
npartitions=17                                                                                                                                     
                    int64  object  object     int64  int64  int64  float64  object  object  object  object  object  float64      object      object
                      ...     ...     ...       ...    ...    ...      ...     ...     ...     ...     ...     ...      ...         ...         ...
...                   ...     ...     ...       ...    ...    ...      ...     ...     ...     ...     ...     ...      ...         ...         ...
                      ...     ...     ...       ...    ...    ...      ...     ...     ...     ...     ...     ...      ...         ...         ...
                      ...     ...     ...       ...   

In [6]:
comp = pickle.load(comp_pickle)
comp.columns

<bound method DataFrame.info of          gvkey   datadate  fyearq  fqtr   fyr indfmt consol popsrc datafmt  \
0       001004 1971-08-31  1971.0   1.0   5.0   INDL      C      D     STD   
1       001004 1971-11-30  1971.0   2.0   5.0   INDL      C      D     STD   
2       001004 1972-02-29  1971.0   3.0   5.0   INDL      C      D     STD   
3       001004 1972-05-31  1971.0   4.0   5.0   INDL      C      D     STD   
4       001004 1972-08-31  1972.0   1.0   5.0   INDL      C      D     STD   
...        ...        ...     ...   ...   ...    ...    ...    ...     ...   
653590  345980 2020-03-31  2020.0   1.0  12.0   INDL      C      D     STD   
653591  345980 2020-06-30  2020.0   2.0  12.0   INDL      C      D     STD   
653592  345980 2020-09-30  2020.0   3.0  12.0   INDL      C      D     STD   
653593  345980 2020-12-31  2020.0   4.0  12.0   INDL      C      D     STD   
653594  345980 2021-03-31  2021.0   1.0  12.0   INDL      C      D     STD   

         tic  ...    cshoq    d

In [12]:
drp_crsp=['DIVAMT','EXCHCD','FACPR', 'NWPERM', 'BIDLO', 'ASKHI','TICKER','BID','ASK','SPREAD','RETX']
crsp.drop(columns=drp, inplace=True)

In [19]:
crsp['PRC']=crsp['PRC'].abs()

,Unnamed: 0,PERMNO,date,NCUSIP,TICKER,CUSIP,DIVAMT,PRC,VOL,RET,BID,ASK,SHROUT,SPREAD,RETX
0,1,10000,1985-12-31,NaN,NaN,68391610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,10000,1986-01-31,68391610,OMFGA,68391610,NaN,4.3750,1771.0,NaN,NaN,NaN,3680.0,0.250,NaN
2,3,10000,1986-02-28,68391610,OMFGA,68391610,NaN,3.2500,828.0,-0.257143,NaN,NaN,3680.0,0.250,-0.257143
3,4,10000,1986-03-31,68391610,OMFGA,68391610,NaN,4.4375,1078.0,0.365385,NaN,NaN,3680.0,0.125,0.365385
4,5,10000,1986-04-30,68391610,OMFGA,68391610,NaN,4.0000,957.0,-0.098592,NaN,NaN,3793.0,0.250,-0.098592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,10001,2017-02-28,36720410,EGAS,36720410,NaN,12.6500,3979.0,0.000000,12.65,12.70,10520.0,NaN,0.000000
396,397,10001,2017-03-31,36720410,EGAS,36720410,0.075,12.7000,6335.0,0.009881,12.60,12.70,10520.0,NaN,0.003953
397,398,10001,2017-04-28,36720410,EGAS,36720410,NaN,12.5000,3906.0,-0.015748,12.50,12.55,10520.0,NaN,-0.015748
398,399,10001,2017-05-31,36720410,EGAS,36720410,NaN,12.7000,4401.0,0.016000,12.65,12.70,10520.0,NaN,0.016000


#### Questions:
- What exactly are the variables we need from IBES (in terms of dates)
- Merging requires too much RAM and I can't just use a part of the dataset, otherwise it won't find matches
- Maybe find the most relevant 2/3 firms for each industry first? Or try replicating Hartzmark and Shue(2018)?
- How to control for T's own earning surprise
- When do we use characteristic-adjusted returns (or an easier version of that)
- When to use Kenneth French's data (market excess return, risk-free rate, SMB, HML, UMD, and short term reversal portfolios as well as size cutoffs)
- At some point we also have to adjust for mechanical relation: 
    - We remove from the characteristic-matched portfolio a stock's own return and the return of firms included in the calculation of surpriset-1.

##### To measure return on day t:
"Our measure of return on day t is a stock's raw return on day t minus the day t return of this characteristic-matched portfolio."

##### To measure earnings surprise:
we take each analyst's most recent forecast, thereby limiting the sample to only one forecast per analyst, and then take the median of this number within a certain time window for each firm's earnings announcement. In our base specification, we take all analyst forecasts made between two and fifteen days prior to the announcement of earnings.

Calculation:
\begin{equation*}
surprise\it = \frac{actrualEarnings - medianEstimate_i[t-15,t-2]}{price_i,t-3}
\end{equation*}

In [26]:
comp.columns

Index(['gvkey', 'datadate', 'fyearq', 'fqtr', 'fyr', 'tic', 'cusip',
       'datacqtr', 'datafqtr', 'actq', 'atq', 'ceqq', 'cshoq', 'dlcq', 'dlttq',
       'ibq', 'revtq', 'exchg', 'costat', 'prccq', 'naics', 'sic'],
      dtype='object')

In [25]:
drop_comp=['indfmt', 'consol','curcdq','popsrc','datafmt','tic','datacqtr', 'datafqtr', 'actq','ceqq','dlcq', 'dlttq','exchg', 'costat']
comp.drop(columns=drop_comp,inplace=True)

C:\Users\pedri\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,gvkey,datadate,fyearq,fqtr,fyr,tic,cusip,datacqtr,datafqtr,actq,...,cshoq,dlcq,dlttq,ibq,revtq,exchg,costat,prccq,naics,sic
0,001004,1971-08-31,1971.0,1.0,5.0,AIR,000361105,1971Q3,1971Q1,NaN,...,0.780,1.998,2.378,0.150,5.185,11.0,A,14.999987,423860,5080
1,001004,1971-11-30,1971.0,2.0,5.0,AIR,000361105,1971Q4,1971Q2,NaN,...,0.780,2.708,1.902,0.185,5.515,11.0,A,19.249980,423860,5080
2,001004,1972-02-29,1971.0,3.0,5.0,AIR,000361105,1972Q1,1971Q3,NaN,...,1.009,2.982,2.184,0.204,6.160,11.0,A,25.624976,423860,5080
3,001004,1972-05-31,1971.0,4.0,5.0,AIR,000361105,1972Q2,1971Q4,12.62,...,1.034,2.967,2.291,0.315,8.219,11.0,A,32.999938,423860,5080
4,001004,1972-08-31,1972.0,1.0,5.0,AIR,000361105,1972Q3,1972Q1,NaN,...,1.044,3.118,2.085,0.351,7.555,11.0,A,21.749983,423860,5080


In [29]:
#We have to drop he last digit in CUSIP for Compustat to be equal to CRSP
comp['cusip'] = comp['cusip'].str[:-1]
comp

,gvkey,datadate,fyearq,fqtr,fyr,tic,cusip,datacqtr,datafqtr,actq,...,cshoq,dlcq,dlttq,ibq,revtq,exchg,costat,prccq,naics,sic
0,001004,1971-08-31,1971.0,1.0,5.0,AIR,00036110,1971Q3,1971Q1,NaN,...,0.780,1.998,2.378,0.150,5.185,11.0,A,14.999987,423860,5080
1,001004,1971-11-30,1971.0,2.0,5.0,AIR,00036110,1971Q4,1971Q2,NaN,...,0.780,2.708,1.902,0.185,5.515,11.0,A,19.249980,423860,5080
2,001004,1972-02-29,1971.0,3.0,5.0,AIR,00036110,1972Q1,1971Q3,NaN,...,1.009,2.982,2.184,0.204,6.160,11.0,A,25.624976,423860,5080
3,001004,1972-05-31,1971.0,4.0,5.0,AIR,00036110,1972Q2,1971Q4,12.62,...,1.034,2.967,2.291,0.315,8.219,11.0,A,32.999938,423860,5080
4,001004,1972-08-31,1972.0,1.0,5.0,AIR,00036110,1972Q3,1972Q1,NaN,...,1.044,3.118,2.085,0.351,7.555,11.0,A,21.749983,423860,5080


- if there are duplicates just remove them (make sure there are no duplicates)
- 

##### Next steps for CRSP
- drop the rest of the variables that are unnecessary


##### Next steps for IBES
- annadats_act -> the day of the announcement
- remove estimator, fpi, revdats, revtims
- t -> anndats - anndats_act >= -15
- filter the analysts that didn't follow this
- by ticker fpedats analys: sort anndats and use value at the most recent anndats 
- take the median/mean of the leftover forecasts
- consensus forecast = mean or median across 6 analysts for a given ticker at a given fpedats 
- by ticker fpedats: unique obs -> make it for firm level

##### Next steps for Compustat
- ibes and compustat are quarterly and crsp is monthly
- fyearq (year) fqtr(qtr)
- cusip in compustat is the same as the cusip in crsp
- in compusat we must drop the last digit in cusip 
- 'prccq' is the book value per share
- 'cshoq' number of shares outstanding
- ibq -> NI
- revtq -> total revenue (can be used for firm size)
- 'atq' -> total assets

In [34]:
drop_ibes=['TICKER','ESTIMATOR','FPI','REVDATS','REVTIMS']
ibes.drop(columns=drop_ibes,inplace=True)
ibes

,Unnamed: 0,CUSIP,ANALYS,VALUE,FPEDATS,ANNDATS,ANNTIMS,ACTUAL,ANNDATS_ACT,ANNTIMS_ACT
0,1,87482X10,71182,0.20,2014-01-31,2014-03-10,1960-01-01 00:20:00,NaN,NaN,NaN
1,2,87482X10,119962,0.30,2014-03-31,2014-03-09,1960-01-01 17:05:00,0.12,2014-05-06,1960-01-01 10:45:00
2,3,87482X10,79092,0.37,2014-03-31,2014-03-10,1960-01-01 00:25:00,0.12,2014-05-06,1960-01-01 10:45:00
3,4,87482X10,80474,0.35,2014-03-31,2014-03-10,1960-01-01 06:48:00,0.12,2014-05-06,1960-01-01 10:45:00
4,5,87482X10,50789,0.23,2014-03-31,2014-03-10,1960-01-01 07:45:00,0.12,2014-05-06,1960-01-01 10:45:00
...,...,...,...,...,...,...,...,...,...,...
395,396,26878510,22290,0.14,2015-03-31,2015-04-22,1960-01-01 16:27:00,0.13,2015-04-29,1960-01-01 16:15:00
396,397,26878510,159934,0.10,2015-03-31,2015-04-22,1960-01-01 06:17:00,0.13,2015-04-29,1960-01-01 16:15:00
397,398,26878510,78506,0.10,2015-03-31,2015-04-23,1960-01-01 01:30:00,0.13,2015-04-29,1960-01-01 16:15:00
398,399,26878510,18677,0.14,2015-03-31,2015-04-23,1960-01-01 06:05:00,0.13,2015-04-29,1960-01-01 16:15:00


In [37]:
ibes['ANNTIMS']=ibes['ANNTIMS'].str[10:]
ibes['ANNTIMS_ACT']=ibes['ANNTIMS_ACT'].str[10:]
ibes

,Unnamed: 0,CUSIP,ANALYS,VALUE,FPEDATS,ANNDATS,ANNTIMS,ACTUAL,ANNDATS_ACT,ANNTIMS_ACT
0,1,87482X10,71182,0.20,2014-01-31,2014-03-10,00:20:00,NaN,NaN,NaN
1,2,87482X10,119962,0.30,2014-03-31,2014-03-09,17:05:00,0.12,2014-05-06,10:45:00
2,3,87482X10,79092,0.37,2014-03-31,2014-03-10,00:25:00,0.12,2014-05-06,10:45:00
3,4,87482X10,80474,0.35,2014-03-31,2014-03-10,06:48:00,0.12,2014-05-06,10:45:00
4,5,87482X10,50789,0.23,2014-03-31,2014-03-10,07:45:00,0.12,2014-05-06,10:45:00
...,...,...,...,...,...,...,...,...,...,...
395,396,26878510,22290,0.14,2015-03-31,2015-04-22,16:27:00,0.13,2015-04-29,16:15:00
396,397,26878510,159934,0.10,2015-03-31,2015-04-22,06:17:00,0.13,2015-04-29,16:15:00
397,398,26878510,78506,0.10,2015-03-31,2015-04-23,01:30:00,0.13,2015-04-29,16:15:00
398,399,26878510,18677,0.14,2015-03-31,2015-04-23,06:05:00,0.13,2015-04-29,16:15:00


In [40]:
#transform both columns into datetime.date objects
ibes['ANNDATS']=pd.to_datetime(ibes['ANNDATS'])
ibes['ANNDATS_ACT']=pd.to_datetime(ibes['ANNDATS_ACT'])
#the date.days attribute will return the days in int format
ibes_fltr = ibes[(ibes['ANNDATS']-ibes['ANNDATS_ACT']).dt.days >= -15]

In [41]:
ibes_fltr

,Unnamed: 0,CUSIP,ANALYS,VALUE,FPEDATS,ANNDATS,ANNTIMS,ACTUAL,ANNDATS_ACT,ANNTIMS_ACT
11,12,87482X10,80474,0.100,2014-03-31,2014-05-07,19:32:00,0.12,2014-05-06,10:45:00
14,15,87482X10,80474,0.190,2014-06-30,2014-08-07,23:49:00,0.27,2014-08-06,17:05:00
108,109,26878510,104605,0.804,2013-06-30,2014-02-26,06:36:00,NaN,2014-01-23,17:05:00
109,110,26878510,104605,-0.448,2013-09-30,2014-02-26,06:36:00,NaN,2014-01-23,17:05:00
110,111,26878510,107698,0.070,2013-12-31,2014-02-26,00:16:00,0.20,2014-02-27,23:13:00
111,112,26878510,78506,0.230,2013-12-31,2014-02-26,03:30:00,0.20,2014-02-27,23:13:00
112,113,26878510,147283,0.340,2013-12-31,2014-02-26,07:02:00,0.20,2014-02-27,23:13:00
113,114,26878510,42769,0.200,2013-12-31,2014-02-26,00:07:00,0.20,2014-02-27,23:13:00
114,115,26878510,83355,0.340,2013-12-31,2014-02-26,09:37:00,0.20,2014-02-27,23:13:00
115,116,26878510,137332,0.670,2013-12-31,2014-02-26,05:37:00,0.20,2014-02-27,23:13:00
